In [17]:
#%%
# system level
import os
from os import path
import sys
import argparse


# deep learning
from scipy.stats import pearsonr, spearmanr
import numpy as np
import torch
from torch import nn
from torchvision import models,transforms
import torch.optim as optim
import wandb
from sklearn.model_selection import GroupKFold
from sklearn.linear_model import LinearRegression

# data 
import pandas as pd
import cv2
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import plotly.express as px
import plotly.graph_objects as go
from scipy.optimize import curve_fit

# local
from nerf_qa.DISTS_pytorch.DISTS_pt import DISTS, prepare_image
from nerf_qa.data import LargeQADataset
from nerf_qa.settings import DEVICE_BATCH_SIZE

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')




DATA_DIR = "/home/ccl/Datasets/NeRF-QA"
REF_DIR = path.join(DATA_DIR, "Reference")
SYN_DIR = path.join(DATA_DIR, "NeRF-QA_videos")
SCORE_FILE = path.join(DATA_DIR, "NeRF_VQA_MOS.csv")

# Read the CSV file
scores_df = pd.read_csv(SCORE_FILE)
train_df = pd.read_csv("/home/ccl/Datasets/NeRF-QA-Large-1/scores.csv")

In [8]:
def plot_dists_mos(df):
    x_data = df['MOS']
    y_data = df['DISTS']
    group_data = df['reference_filename']
    def sqrt_func(x, a, b):
        return a + b * x #np.sqrt(x)
    # Fit the model
    params, params_covariance = curve_fit(sqrt_func, x_data, y_data)

    # Predict using the fitted model
    x_range = np.linspace(min(x_data), max(x_data), 400)
    y_pred = sqrt_func(x_range, *params)

    # Plotting
    fig = go.Figure()

    # Color points by group. We need to create a trace for each unique group.
    unique_groups = df['reference_filename'].unique()
    for group in unique_groups:
        group_x = df[df['reference_filename'] == group]['MOS']
        group_y = df[df['reference_filename'] == group]['DISTS']
        fig.add_trace(go.Scatter(x=group_x, y=group_y, mode='markers', name=group))

    # Regression line
    fig.add_trace(go.Scatter(x=x_range, y=y_pred, mode='lines', name='Linear Regression'))

    fig.update_layout(title='Linear Regression between DISTS and MOS',
                      xaxis_title='MOS',
                      yaxis_title='DISTS')
    return fig
fig = plot_dists_mos(scores_df)
display(fig)
display(scores_df.head(3))

,distorted_filename,reference_filename,FVVHD,DISTS,SSIM,GMSD,MOS,DoMOS,DMOS,Buike MOS,...,WaDiQaM_score,CompressVQA,PSNR_Score,MS-SSIM_Score,LPIPS_Score,WaDiQa_score,LPIPS_Score_vgg,PLCC,SROCC,KROCC
0,lego_tensorf.mp4,lego_reference.mp4,9.0069,0.035496,0.99129,0.035686,4.6396,5.000000,-12.1555,5.0,...,0.798469,3.745422,36.646977,0.968563,0.029931,79.846926,0.089946,0.363577,0.474598,0.324468
1,lego_dvgo.mp4,lego_reference.mp4,8.9140,0.025291,0.99086,0.038980,4.5862,4.922538,-10.0190,4.0,...,0.796464,3.848095,36.857570,0.968916,0.026688,79.646396,0.055983,-0.377061,-0.569366,-0.423759
2,lego_tensorf_ss2.mp4,lego_reference.mp4,8.9029,0.037678,0.99005,0.038398,4.3874,4.634879,-2.0850,4.0,...,0.797675,3.738048,36.581300,0.966980,0.031191,79.767545,0.093800,-0.602582,-0.645571,-0.464539


In [23]:


def plot_dists_mos_with_group_regression(df, y_col='DISTS', group_col='reference_filename'):
    # Define a list of colors for the groups
    colors = [
        '#1f77b4',  # Mutated blue
        '#ff7f0e',  # Safety orange
        '#2ca02c',  # Cooked asparagus green
        '#d62728',  # Brick red
        '#9467bd',  # Muted purple
        '#8c564b',  # Chestnut brown
        '#e377c2',  # Raspberry yogurt pink
        '#7f7f7f',  # Middle gray
        '#bcbd22',  # Curry yellow-green
        '#17becf'   # Blue-teal
    ]

    def linear_func(x, a, b):
        return a + b * x

    # Plotting
    fig = go.Figure()

    unique_groups = df[group_col].unique()
    for i, group in enumerate(unique_groups):
        group_df = df[df[group_col] == group]
        group_x = group_df['MOS']
        group_y = group_df[y_col]
        
        # Fit the model for each group
        params, params_covariance = curve_fit(linear_func, group_x, group_y)
        
        # Predict using the fitted model for the group
        x_range = np.linspace(min(group_x), max(group_x), 400)
        y_pred = linear_func(x_range, *params)
        
        # Ensure we use a unique color for each group, cycling through the colors list if necessary
        color = colors[i % len(colors)]
        
        # Data points for the group
        fig.add_trace(go.Scatter(x=group_x, y=group_y, mode='markers', name=f'Data: {group}', marker_color=color))
        
        # Regression line for the group
        fig.add_trace(go.Scatter(x=x_range, y=y_pred, mode='lines', name=f'Regression: {group}', line=dict(color=color)))

    fig.update_layout(title=f'Linear Regression per Group between {y_col} and MOS',
                      xaxis_title='MOS',
                      yaxis_title=y_col)
    return fig

display(plot_dists_mos_with_group_regression(scores_df, 'DISTS'))
display(plot_dists_mos_with_group_regression(train_df, 'DISTS', 'referenced_filename'))

In [14]:

display(plot_dists_mos_with_group_regression(scores_df, 'PSNR_Score'))

In [16]:
display(plot_dists_mos_with_group_regression(scores_df, 'MS-SSIM_Score'))